In [1]:
#!uv pip install datasets langchain_openai langchain

In [24]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from typing import TypedDict, Annotated, Optional
import json
from pathlib import Path
import os

PROVIDER = "openai"

match PROVIDER:
    case "cloud":
        models = {"fast": "Vikhrmodels/Qwen2.5-7B-Instruct-Tool-Planning-v0.1", 
                  "moder": "Qwen/Qwen2.5-Coder-32B-Instruct", 
                  "analysis": "Qwen/Qwen3-235B-A22B-Instruct-2507"}
        print("CLOUD MODEL")
        API_KEY = os.getenv("CLOUD_API_KEY")
        llm = ChatOpenAI(
            model=models["fast"],
            temperature=0.1,
            base_url="https://foundation-models.api.cloud.ru/v1",
            api_key=API_KEY,
        )
    case "openai":
        print("OPENAI MODEL")
        API_KEY = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.0,
            openai_api_key=API_KEY,
        )

OPENAI MODEL


### Датасет персон

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("nvidia/Nemotron-Personas")

#import pandas as pd

#ds = pd.read_csv("marketing_personas_sample.csv")

/home/dsuhoi/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['uuid', 'persona', 'professional_persona', 'sports_persona', 'arts_persona', 'travel_persona', 'culinary_persona', 'skills_and_expertise', 'skills_and_expertise_list', 'hobbies_and_interests', 'hobbies_and_interests_list', 'career_goals_and_ambitions', 'sex', 'age', 'marital_status', 'education_level', 'bachelors_field', 'occupation', 'city', 'state', 'zipcode', 'country'],
        num_rows: 100000
    })
})

### Тест на эмоции в промптах

In [46]:
person_simulation_schema = {
    "title": "person-classifier",
    "description": "Ты - система для валидации характеристик и типов личности, указанных в блоке PERSON! \
    Ты выделяешь наиболее точное описание характера (промпт-характера) и НЕ используешь обобщенных терминов, которые свойственны всем!",
    "type": "object",
    "properties": {
        "emotions": {
            "description": "Список из названий составляющих промпт-характера личности описанной в блоке PERSON (не более 3 элементов)",
            "type": "array",
            "items": {
                "description": "Название типа личности",
                "type": "string",
                #"enum": ["", "", ""]
            },
            "minItems": 1,
            "maxItems": 3
        },
        "mbti": {
            "description": "Тип личности по MBTI теории (Типология Майерс — Бриггс).",
            "type": "string"
        }
        #"reasoning": {
        #    "description": "Обоснование выбранного набора эмоций (не более 1 предложения).",
        #    "type": "string"
        #}
    },
    "required": ["emotions", "mbti"]#, "reasoning"],
}

prompt = ChatPromptTemplate.from_messages([("user", "{person}")])

def ds2prompt(data: dict[str, str]):
    return f"""PERSON:\n{data['persona']}\n###\nAGE: {data['age']}, SEX: {data['sex']}###"""

character_llm = (
    {"person": lambda x: ds2prompt(x["person"])}
    | prompt
    | llm.with_structured_output(person_simulation_schema)
)

In [47]:
N = 18130

In [48]:
%%time
print(character_llm.invoke({"person": ds['train'][N]}))

{'mbti': 'ISFJ', 'emotions': ['Curious', 'Organized', 'Creative']}
CPU times: user 41.8 ms, sys: 3.49 ms, total: 45.2 ms
Wall time: 1.82 s


In [41]:
%%time
print(summary_person_llm.invoke({"person": ds['train'][N]}).content)

Меня зовут Мелисса, мне 57 лет, и я живу на Среднем Западе. Я замужем и имею степень бакалавра в области образования. В своей карьере я работала учителем, создавая инклюзивные учебные среды и разрабатывая увлекательные уроки для разнообразных учеников. Я увлекаюсь искусством, посещаю художественные галереи и организую книжные клубы, где обсуждаю произведения таких авторов, как Зади Смит и Фрида Кало. В свободное время я занимаюсь садоводством, что помогает мне находить вдохновение для творчества и уединения. Я также люблю путешествовать, планируя семейные поездки с культурными и расслабляющими моментами.
CPU times: user 11.7 ms, sys: 2.9 ms, total: 14.6 ms
Wall time: 5.92 s


### Примеры описание схем взаимодействия агентов

In [36]:
person_simulation_schema = {
    "title": "person-simulation",
    "description": "Отвечай на вопрос из INPUT блока с учетом характера персоны из блока PERSONS и его особенностями максимально реалистично!",
    "type": "object",
    "properties": {
        "answer": {
            "description": "Твой ответ на вопрос из INPUT блока.",
            "type": "string"
        },
        "reasoning_question": {
            "description": "Твои мысли на счет самого вопроса. Как ты к нему относишься и хочешь ли на него отвечать.",
            "type": "string"
        },
        "reasoning_answer": {
            "description": "Твои мысли на счет ответа (то, что ты не сказал, но подумал).",
            "type": "string"
        },
        "emotion": {
            "description": "Твои эмоции при ответе на вопрос. Краткое описание в 1 предложение.",
            "type": "string"
        }
    },
    "required": ["answer", "reasoning_answer", "emotion"],
}

prompt = ChatPromptTemplate.from_messages([("system", "Ты - человек, обладающих следующими свойствами (ВСЕГДА СЛЕДУЙ ИМ): {person}"), ("user", "INPUT: {input}")])

def ds2prompt(data: dict[str, str]):
    return f"""PERSON:\n{data['persona']}\n###\nAGE: {data['age']}, SEX: {data['sex']}, MERITAL_STATUS: {data['marital_status']}, EDUCATION_LEVEL: {data['education_level']}\n###\nHOBBIES:\n{data['hobbies_and_interests_list']}\n###\nSKILLS:\n{data['skills_and_expertise']}\n\nSKILLS_LIST: {data['skills_and_expertise_list']}\n###\nPROFESSIONAL:\n{data['professional_persona']}\n###\nCAREER:\n{data['career_goals_and_ambitions']}\n###\nTRAVEL:\n{data['travel_persona']}\n###\nARTS:\n{data['arts_persona']}\n
    ###"""

person_llm = (
    {"input": lambda x: x["input"], "person": lambda x: ds2prompt(x["person"])}
    | prompt
    | llm.with_structured_output(person_simulation_schema)
)

In [37]:
sum_prompt = ChatPromptTemplate.from_messages([("system", "Ты - человек, обладающих следующими свойствами: {person}"), ("user", "Опиши свою краткую биографию не более чем в 5-6 предложений.")])

summary_person_llm = (
    {"person": lambda x: ds2prompt(x["person"])}
    | sum_prompt | llm
)

In [79]:
print(ds2prompt(ds['train'][N]))

PERSON:
Dominika, a well-organized yet flexible 44-year-old, balances curiosity with practicality, preferring small gatherings and solitude, yet actively contributes to her community through volunteering.
###
AGE: 44, SEX: Female, EDUCATION_LEVEL: graduate
###
HOBBIES:
['hiking', 'reading (non-fiction, history, science)', 'book clubs', 'yoga', 'community service']
###
SKILLS:
As a Project Management Specialist, Dominika is well-versed in agile methodologies, risk assessment, and stakeholder communication. She's proficient in project management software like JIRA and Asana, and has a knack for balancing ambitious goals with practical constraints. Her graduate degree in business has equipped her with strong analytical skills and a solid understanding of finance and marketing principles.

SKILLS_LIST: ['agile methodologies', 'risk assessment', 'stakeholder communication', 'project management software (jira, asana)', 'analytical skills', 'financial and marketing principles']
###
PROFESSION

### Тестирование виртуальных пользователей

In [66]:
N = 18752

In [16]:
%%time
print(summary_person_llm.invoke({"person": ds['train'][N]}).content)

NotFoundError: Error code: 404

In [68]:
%%time
question = "Как вы относитесь к звонкам-опросам после обращения в контактный центр для решения проблемы с интернетом на личном устройстве?"

input_data = {"input": question, "person": ds["train"][N]}

result = person_llm.invoke(input_data)
result

CPU times: user 36.5 ms, sys: 5.29 ms, total: 41.8 ms
Wall time: 1.54 s


{'response': 'Я понимаю, что такие звонки могут быть неудобными, но они помогают улучшить качество обслуживания. Как правило, я стараюсь быть терпеливой и сотрудничественной во время таких звонков.',
 'target_skills': ['communication', 'patience', 'customer service'],
 'tone': 'нейтрально',
 'reasoning_question': 'Можете ли вы рассказать об одном случае, когда вам приходилось общаться с представителем службы поддержки?',
 'examiner_emotion': 'нейтрально',
 'adaptation_notes': 'Переход к более конкретному примеру общения с представителем службы поддержки.'}

In [9]:
result

{'answer': 'Как опытная медсестра, я понимаю важность обратной связи и улучшения качества обслуживания. Звонки-опросы могут быть полезны для получения информации о том, как мы можем улучшить свою работу. Однако важно, чтобы эти звонки не мешали решению текущей проблемы клиента. Если звонок-опрос не отнимает много времени и помогает улучшить качество обслуживания, то я считаю это полезным.',
 'reasoning_question': 'Вы хотите узнать мое мнение о звонках-опросах после обращения в контактный центр?',
 'reasoning_answer': 'Я думаю, что звонки-опросы могут быть полезными для улучшения качества обслуживания, но они должны не мешать решению текущей проблемы клиента.',
 'emotion': 'положительная'}

# Построение системы на хакатон

## Блок с системой оценки прохождения тестов

### Описание схем и агентов

In [31]:
def person2prompt(data: dict[str, str]):
    return f"""PERSON:
{data['persona']}
####
SEX: {data['sex']} # AGE: {data['age']}
####
SKILLS:
{data['skills_and_expertise']}
####
PROFESSIONAL:
{data['professional_persona']}
####
HOBBIES:
{data['hobbies_and_interests']}
"""

In [6]:
def test_info2prompt(test_info: dict[str, str | dict], with_criterions: bool=True) -> str:
    criterions = ""
    if with_criterions:
        criterions += "###Criterions:\n"
        criterions += "\n##\n".join([f"Type: {name}\nDescription: {desc}\n" for name, desc in test_info["criterions"].items()])
    
    return f"""###Title: {test_info["title"]}\n
###Description:\n {test_info["description"]}\n
###Target:\n {test_info["target"]}\n
{criterions}
"""

def dialog_history2prompt(dialog_history: dict[str, str | dict], with_comments: bool=False) -> str:
    result = ""
    from_convert = {"user": "(U)", "assistant": "(A)"}
    
    for block in dialog_history:
        from_type = from_convert[block['from']]
        result += f"#{from_type}: {block["content"]}"
        if with_comments and from_type == "(A)":
            result += f"[{block['meta'].get('comments', '')}]"
        result += "#\n\n"
    return result

def expert_results2prompt(expert_results: dict[str, str|int]) -> str:
    return "\n##\n".join([f"CRITERION: {name}\nSCORE: {data['score']}\nJustification: {data['reasoning']}" 
                          for name, data in expert_results.items()])

In [142]:
main_judge_schema = {
    "title": "main-judge",
    "description": "Ответ агента-оценщика прохождения теста (TEST_BLOCK) на основе результатов агентов-аналитиков (EXPERTS_BLOCK).",
    "type": "object",
    "properties": {
        "main_score": {
            "description": "Общая оценка на основе ответов из EXPERTS_BLOCK. Ответ идет по баллам от 1 до 5.",
            "type": "integer",
            "minimum": 1,
            "maximum": 5
        },
        "score_reasoning": {
            "description": "Описание действий пользователя в совокупности и обоснование оценки.",
            "type": "string",
        },
        "comments": {
            "description": "Общие рекомендации ко всем критериям в совокупности по действиям пользователя на основе анализа агентов-аналитиков.",
            "type": "string"
        }
    },
    "required": ["main_score", "score_reasoning", "comments"]
}

judge_prompt = ChatPromptTemplate.from_messages([("system", """Ты - система оценки пользователя по прохождению теста, суть которого заключается в следующем:
###\n TEST_BLOCK
{test_info}
###
Твоя задача дать краткий ответ по указанной JSON схеме!"""), 
("assistant", """Экспертная оценка по каждому критерию теста представлена ниже от каждого агента:
###\n EXPERTS_BLOCK
{expert_results}
###
Диалог между экзаменатором (A) и пользователем (U) представлен ниже для более полной картины происходящего:
###\n HISTORY_BLOCK
{dialog_history}
###
""")])

judge_agent = {"dialog_history": lambda x: dialog_history2prompt(x["dialog_history"]), 
              "expert_results": lambda x: x["expert_results"], 
              "test_info": lambda x: x["test_info"]} | judge_prompt | llm.with_structured_output(main_judge_schema)

In [143]:
analyst_schema = {
    "title": "analyst",
    "description": "Ответ аналитика по критерию из блока CRITERION по диалогу из блока HISTORY.",
    "type": "object",
    "properties": {
        "score": {
            "description": "Оценка за прохождение теста (TEST_BLOCK) пользователем (U).",
            "type": "integer",
            "minimum": 1,
            "maximum": 10
        },
        "reasoning": {
            "description": "Краткое обоснование поставленной оценки по заданному критерию",
            "type": "string"
        }
    },
    "required": ["score", "reasoning"]
}

analyst_prompt = ChatPromptTemplate.from_messages([("system", """Ты - аналитик качества ответов пользователя (U) в ходе прохождения теста, суть которого заключается в следующем:
###\n TEST_BLOCK
{test_info}
###
Ты опираешься ТОЛЬКО на следующий критерий и даешь свою оценку исключительно по этому признаку из диалога пользователя (U) с экзаменатором (A):
###\n CRITERION
{criterion_descr}
###
"""), ("assistant", """История общения пользователя (U) с экзаменатором (A) [в этой истории в квадратных кавычках могут быть комментарии по действиям пользователя от экзаменатора]:
###\n HISTORY
{dialog_history}
###""")])

def criterion2prompt(x):
    result = ""
    for name, descr in x.items():
        result += f"Type: {name}\nDescription: {descr}"
        break
    return result

analyst_agent = {"dialog_history": lambda x: dialog_history2prompt(x["dialog_history"], with_comments=True), 
              "criterion_descr": lambda x: criterion2prompt(x["criterion_descr"]), 
              "test_info": lambda x: test_info2prompt(x["test_info"], with_criterions=False)} | analyst_prompt | llm.with_structured_output(analyst_schema)

### Валиадция системы оценки тестирования

In [4]:
data = {
  "test_info": {
    "title": "Перенос срока релиза по просьбе команды: переговоры с бизнес-стейкхолдером",
    "description": "Проверяется умение кандидата аргументированно управлять ожиданиями, коммуницировать риски и договариваться о переносе срока без потери доверия.",
    "target": "Оценить навыки управления ожиданиями, ясность коммуникации, работу с рисками, переговоры и фиксацию договорённостей.",
    "criterions": {
      "Expectation Management": "Корректная установка и корректировка ожиданий стейкхолдера; отсутствие чрезмерных обещаний.",
      "Risk Communication": "Чёткое и правдивое описание рисков и их влияния; указание на вероятности/неопределённости.",
      "Negotiation & Options": "Предложение реалистичных вариантов, компромиссов и критериев принятия решения.",
      "Clarity & Next Steps": "Структура сообщения, конкретные шаги, сроки, ответственные и каналы подтверждений.",
      "Empathy & Tone": "Уважительный, спокойный тон, признание приоритетов бизнеса и ограничений команды."
    }
  },

  "dialog_history": [
    {
      "from": "assistant",
      "content": "Нам важен релиз в следующую пятницу. Это публично озвучено клиентам. Сможете подтвердить сроки?",
      "meta": {}
    },
    {
      "from": "user",
      "content": "С текущими багами по платежам есть риск не уложиться к пятнице. Предлагаю рассмотреть перенос на понедельник, чтобы не публиковать нестабильную сборку.",
      "meta": {}
    },
    {
      "from": "assistant",
      "content": "Перенос звучит болезненно. Какие у нас варианты, кроме сдвига даты?",
      "meta": {}
    },
    {
      "from": "user",
      "content": "Вариант 1: релизим только фиксы по безопасности и платежам, без новых фич. Вариант 2: откатываем проблемную часть и оставляем остальное. Оба требуют регресса в выходные.",
      "meta": {}
    },
    {
      "from": "assistant",
      "content": "Какой риск инцидента, если релиз делать в пятницу с урезанным объёмом?",
      "meta": {}
    },
    {
      "from": "user",
      "content": "Оценочно 15–20% шанс на деградацию платежей при высокой нагрузке. Можем снизить до ~5–7%, если провести нагрузочные тесты в субботу и зарезервировать on-call.",
      "meta": {}
    },
    {
      "from": "assistant",
      "content": "Кто будет на on-call и как сообщим клиентам об изменениях в составе релиза?",
      "meta": {}
    },
    {
      "from": "user",
      "content": "Пока не назначал on-call и коммуникацию, сначала хотел согласовать стратегию. Если окей вариант с урезанным релизом, оформлю план в Confluence и разошлю рассылку.",
      "meta": {
        "comments": "Критично: отсутствие заранее подготовленных ответственных и плана коммуникаций при уже предложенной стратегии."
      }
    },
    {
      "from": "assistant",
      "content": "Понял. Тогда фиксируем: в пятницу выпускаем только обязательные фиксы, в субботу нагрузочные тесты, on-call — кто именно, сроки и каналы оповещения?",
      "meta": {}
    },
    {
      "from": "user",
      "content": "Назначаю on-call: Иванов и Петров, 24/7 до понедельника 10:00. Сегодня до 17:00 пришлю план с рисками, сценариями отката и шаблоном письма клиентам. Приму на себя коммуникацию с ключевыми клиентами.",
      "meta": {}
    }
  ]
}
test_info = data["test_info"]
dialog_history = data["dialog_history"]

In [149]:
# pip install langgraph langchain pydantic
from __future__ import annotations
from typing import TypedDict, Dict, Any, List
from langgraph.graph import StateGraph, END

# === ВАЖНО ===
# Считаем, что ВЫ уже где-то выше в коде создали:
# - analyst_agent: Chain   (ожидает keys: dialog_history, criterion_descr, test_info)
# - judge_agent:   Chain   (ожидает keys: dialog_history, expert_results, test_info)
#
# А также у вас есть оригинальные хелперы (если нет — ниже есть локальный вариант для EXPERTS_BLOCK):
# - dialog_history: raw [{"from": "...", "content": "...", "meta": {...}}, ...]
# - test_info: raw {"title":..., "description":..., "target":..., "criterions": {...}}
#
# Требование по комментариям:
# В dialog_history комментарии оставляет только assistant и только по КРИТИЧНЫМ ошибкам пользователя.


# --- Если у вас отсутствует helper, который форматирует EXPERTS_BLOCK ---
def expert_results2prompt_local(expert_results: Dict[str, Dict[str, Any]]) -> str:
    """
    Формирует EXPERTS_BLOCK для judge_agent из словаря:
    {
      "CriterionName": {"score": int, "reasoning": str}, ...
    }
    """
    blocks = []
    for name, data in expert_results.items():
        score = data.get("score")
        reasoning = data.get("reasoning", "")
        blocks.append(f"CRITERION: {name}\nSCORE: {score}\nJustification: {reasoning}")
    return "\n##\n".join(blocks)


# --- Состояние графа ---
class EvalState(TypedDict, total=False):
    # Вход
    test_info: Dict[str, Any]            # {"title", "description", "target", "criterions": {name: descr}}
    dialog_history: List[Dict[str, Any]] # [{"from": "assistant"/"user", "content": "...", "meta": {...}}, ...]
    # Промежуточные
    expert_results: Dict[str, Dict[str, Any]] | None = None  # {"criterion_name": {"score": int, "reasoning": str}}
    experts_block: str | None = None                  # EXPERTS_BLOCK для judge_agent
    # Выход
    main_judge: Dict[str, Any] | None = None          # {"main_score": int, "score_reasoning": str, "comments": str}


# --- Узел: запуск аналитиков по каждому критерию ---
def run_analysts(state: EvalState) -> EvalState:
    test_info = state["test_info"]
    dialog_history = state["dialog_history"]

    criterions: Dict[str, str] = test_info.get("criterions", {})
    expert_results: Dict[str, Dict[str, Any]] = {}
    print("Агенты-аналитики за работой!")
    for name, descr in criterions.items():
        # ВАЖНО: analyst_agent у вас уже содержит маппинг
        # ("dialog_history" -> dialog_history2prompt(with_comments=True), 
        #  "criterion_descr" -> criterion2prompt, 
        #  "test_info" -> test_info2prompt(with_criterions=False))
        payload = {
            "dialog_history": dialog_history,
            "criterion_descr": {name: descr},
            "test_info": test_info
        }
        result = analyst_agent.invoke(payload)  # -> {"score": int, "reasoning": str}
        expert_results[name] = {
            "score": result["score"],
            "reasoning": result["reasoning"]
        }
        print(f"Отработал {name} слоняра:\n{result}\n\n")

    return {**state, "expert_results": expert_results}


# --- Узел: подготовка EXPERTS_BLOCK для судьи ---
def build_experts_block(state: EvalState) -> EvalState:
    expert_results = state["expert_results"]
    experts_block = expert_results2prompt_local(expert_results)
    return {**state, "experts_block": experts_block}


# --- Узел: финальная оценка главным судьёй ---
def run_judge(state: EvalState) -> EvalState:
    payload = {
        "dialog_history": state["dialog_history"],  # judge_agent сам вызовет dialog_history2prompt (без комментариев)
        "expert_results": state["experts_block"],   # уже готовый EXPERTS_BLOCK (строка)
        "test_info": state["test_info"]
    }
    main_judge = judge_agent.invoke(payload)  # -> {"main_score": int, "score_reasoning": str, "comments": str}
    return {**state, "main_judge": main_judge}


# --- Сборка графа ---
def build_graph():
    graph = StateGraph(EvalState)
    graph.add_node("analysts", run_analysts)
    graph.add_node("experts_prepare", build_experts_block)
    graph.add_node("judge", run_judge)

    graph.set_entry_point("analysts")
    graph.add_edge("analysts", "experts_prepare")
    graph.add_edge("experts_prepare", "judge")
    graph.add_edge("judge", END)
    return graph.compile()


# === Пример запуска ===
# (ниже минимальный пример; подайте сюда ваши реальные test_info и dialog_history)
judgment_subsystem = build_graph()

In [152]:
# Пример test_info и dialog_history должны соответствовать вашим форматам.

init_state: EvalState = {
    "test_info": test_info,
    "dialog_history": dialog_history
}

final_state = judgment_subsystem.invoke(init_state)
# Финальный JSON от главного судьи:
print(final_state["main_judge"])

Агенты-аналитики за работой!
Отработал Expectation Management слоняра:
{'score': 3, 'reasoning': 'Кандидат демонстрирует способность аргументированно управлять ожиданиями и коммуницировать риски. Однако, он не полностью фиксирует все условия переноса срока релиза, что может вызвать недопонимание в дальнейшем.'}


Отработал Risk Communication слоняра:
{'score': 3, 'reasoning': 'Кандидат четко описывает риски и их влияние, указывая на вероятности и неопределенности. Однако, он не полностью фиксирует все условия переноса релиза, такие как конкретные сроки и каналы оповещения клиентов.'}


Отработал Negotiation & Options слоняра:
{'score': 3, 'reasoning': 'Кандидат предложил несколько вариантов решения проблемы и оценил риски каждого из них. Однако, он не предложил реалистичных вариантов компромисса или критериев принятия решения.'}


Отработал Clarity & Next Steps слоняра:
{'score': 3, 'reasoning': 'Ответ пользователя структурирован и содержит конкретные шаги для выполнения задачи. Однако

## Система генерации промптов для агентов-экзаменаторов

### Схемы мета-агентов

In [24]:
def get_person_info(person_dataset, person_id: int | None = None, structured: bool = False) -> str | dict[str, str]:
    person = person_dataset.iloc[person_id] if person_id else person_dataset.sample().iloc[0]
    person = person.to_dict()
    return person if structured else person2prompt(person)

In [44]:
meta_person_schema = {
"title": "person-generator",
    "description": "Ответ системы генерации личности агента-экзаменатора.",
    "type": "object",
    "properties": {
        "character_info": {
            "description": "Новое краткое описание личности агента-экзаменатора, которое имеет вид структурированного промпт-характера (описание типов личности) для имитации поведения без лишних фактов.",
            "type": "string",
        },
    },
    "required": ["character_info"]
}
meta_person_prompt = ChatPromptTemplate.from_messages([("system", """Ты - система для генерации личности агента-экзаменатора, которая базируется на следующем описании человека:
### PERSON_BLOCK\n
{person}
###
Твоя главная задача - выявить особенности характера текущего описания изменить детали личности, 
убрав несущественные факты, для соответствия данного описания человеку, который мог бы походить на роль экзаменатора.
- Делай описания личности небольшим (1-3 коротких предложения)!
- Не переноси напрямую все факты из PERSON_BLOCK!
- Добавь промпт-характеру персонажа подробное описание типов характера!
- Описания должны иметь стиль сводок - только четкая структура и минимум бесполезных фактов!
""")])

meta_person_agent = {"person": lambda x: person2prompt(x["person"])} | meta_person_prompt | llm.with_structured_output(meta_person_schema)

In [62]:
meta_advisor_schema = {
"title": "advisor-generator",
    "description": "Ответ системы генерации агентов-экзаменаторов из информации о тесте (TEST_BLOCK)",
    "type": "object",
    "properties": {
        "main_plan": {
            "description": "Из описания теста выдели ключевой набор тем, которые должен раскрыть агент (по порядку)",
            "type": "array",
            "minItems": 2,
            "maxItems": 5,
            "items": {
                "description": "Название темы и её суть (1 предложение)",
                "type": "string"
            }
        },
        "test_info": {
            "description": "Важная нформации/контекста тестирования со стороны тестируемого пользователя (TEST_BLOCK).\
            Т.е. та необходимая информация, которая должна быть важна для экзаменатора по отношению к тесту, который будет проходить пользователь (не более 2-3 предложений).",
            "type": "string"
        },
    },
    "required": ["main_plan", "test_info"]
}

meta_advisor_prompt = ChatPromptTemplate.from_messages([("system", """Ты - система подготовки контекста для экзаменаторов на основе информации о тесте:
###\n TEST_BLOCK
{test_info}
### \n PERSON_BLOCK (информация о личности экзаменатора)\n
{person}
###
Тебе нужно помочь экзаменатору подготовиться к тестированию с пользователем и важно правильно составить JSON c контекстом!""")])

meta_advisor_agent = {"test_info": lambda x: test_info2prompt(x["test_info"]),
                     "person": lambda x: x["person"]} | meta_advisor_prompt | llm.with_structured_output(meta_advisor_schema)

### Валидация системы генерации агентов

In [56]:
person = get_person_info(ds, person_id=1051, structured=True)

person_result = meta_person_agent.invoke({"person": person})
print(person_result)

{'character_info': 'Мужчина в возрасте 68 лет, бывший организатор мероприятий и волонтерский координатор. Он обладает сильными организационными навыками и практическим мышлением, что делает его отличным организатором. Он любит объединять людей и обеспечивать выполнение всех задач в срок. Его открытость к новым идеям позволяет ему эффективно использовать различные инструменты для планирования мероприятий и коммуникации. Он также увлечен историей и часто посещает музеи и исторические места с женой.'}


In [55]:
results = meta_advisor_agent.invoke({"test_info": test_info, "person": person_result["character_info"]})
print(results)

{'main_plan': ['Перенос срока релиза по просьбе команды', 'Управление ожиданиями бизнес-стейкхолдера', 'Коммуникация рисков и переговоры', 'Фиксация договоренностей и следующих шагов'], 'test_info': 'Экзаменатор должен проверить умение кандидата аргументированно управлять ожиданиями, коммуницировать риски и договариваться о переносе срока без потери доверия.'}


## Система тестирования пользователя системы (User + Advisor + Assistant часть)

### Описание схем агентов

In [78]:
from typing import List, Dict, Any, Optional, TypedDict, Literal
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

In [63]:
def plans2prompt(plans: list[str]) -> str:
    """
    Ожидается список тем: []
    """
    return "\n".join([f"{i}) {p}" for i, p in enumerate(plans, start=1)] )
    

def dialog2prompt(messages: list[dict[str, str]]) -> str:
    """
    Ожидается список вида: [{"role":"user"/"assistant", "content":"..."} , ...]
    """
    lines = []
    for m in messages:
        role = m.get("role", "user")
        content = m.get("content", "")
        lines.append(f"{role.upper()}: {content}")
    return "\n".join(lines)

In [65]:
advisor_schema = {
    "title": "advisor",
    "description": "Ответ экзаменатора в ходе общения с пользователем с учетом проделанного плана.",
    "type": "object",
    "properties": {
        "response": {
            "description": "Реалистичный ответ пользователя от лица экзаменатора на основе истории общения с ним и планом тестирования (PLAN_BLOCK).",
            "type": "string",
        },
        "comment": {
            "description": "Комментарий со стороны экзаменатора на крайний/последний ответ пользователя в случае очень хорошего или очень плохого ответа. По умолчанию это поле: null",
            "type": ["string", "null"]
        },
        "realised_plan": {
            "description": "Данное поле заполняется True, ТОЛЬКО если предыдущий ответ пользователя можно считать завершением темы актуального плана.",
            "type": "boolean"
        }
    },
    "required": ["response", "comment", "realised_plan"]
}
advisor_prompt = ChatPromptTemplate.from_messages([("system", """Ты - Экзаменатор (assistant) со следующим описанием личности:
### PERSON_BLOCK\n
{person}
###
Краткая информация о тесте:
### TEST_BLOCK\n
{test_info}
###

Актуальные темы по порядку (раскрываем текущую первую тему с учетом истории общения с пользователем):
###
{plans}
###

Твоя главная задача - провести тестирование пользователя (user) поэтапно по плану тестирования и в случае реализации всех планов тестирования быстрее завершить диалог!
- Строго следуй своим паттернам личности и стилю общения!
- Задавай направление беседы вокруг актуальных тем плана!
- Отмечай поле realised_plan, если актуальная тема общения завершилась на предыдущем сообщении пользователя!
- Отвечай на вопросы пользователя во время теста, если посчитаешь их существенными (иначе игнорируй или проси вернуться к диалогу)!
- Если история общения пустая, то тогда начинай диалог сам и веди к первой теме сразу!
"""), ("assistant", "История общения с пользователем (user) и экзаменатором (assistant):\n{dialog_history}")])

advisor_agent = {"person": lambda x: x["person"], 
                 "test_info": lambda x: x["test_info"], 
                 "plans": lambda x: plans2prompt(x["plans"]),
                 "dialog_history": lambda x: dialog2prompt(x["messages"])} | advisor_prompt | llm.with_structured_output(advisor_schema)

In [66]:
planner_schema = {
    "title": "plan-assistant",
    "description": "Ассистент, который отмечает актуальную тему плана, формирует список 'актуальные+будущие', решает пора ли завершать диалог.",
    "type": "object",
    "properties": {
        "current_plan_index": {
            "description": "Индекс текущей темы плана (0-based). Если план завершен, укажи последний индекс.",
            "type": "integer"
        },
        "active_topic": {
            "description": "Номер текущей активной темы (по порядку в PLAN_BLOCK)",
            "type": "integer",
            "minimum": 1
        },
        "upcoming_topics": {
            "description": "Список следующих тем (по порядку)",
            "type": "array",
            "items": {"type": "string"}
        },
        "should_end_dialog": {
            "description": "True — если все темы раскрыты или пользователь явно завершил общение.",
            "type": "boolean"
        },
        "assistant_comment": {
            "description": "Короткий служебный комментарий в виде подсказки для пользователя, если ему требуется помощь.",
            "type": "string"
        }
    },
    "required": ["current_plan_index", "active_topic", "upcoming_topics", "should_end_dialog", "assistant_comment"]
}

planner_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Ты — Планировщик теста (assistant-planner). Твоя задача — отмечать актуальные темы плана, "
     "обновлять их порядок с учетом истории и сигналить, когда диалог нужно корректно завершать.\n\n"
     "### PERSON_BLOCK\n{person}\n###\n"
     "### TEST_BLOCK\n{test_info}\n###\n"
     "### PLAN_BLOCK (по порядку)\n{full_plan}\n###\n\n"
     "GUIDE:\n"
     "- Проанализируй историю диалога.\n"
     "- Если предыдущий ответ экзаменатора имел realised_plan=True — переходи к следующей теме.\n"
     "- Если пользователь явно запросил завершение или все темы раскрыты — should_end_dialog=True.\n"
     "- Выведи structured output по schema plan-assistant."
    ),
    ("assistant", "История общения (user/assistant):\n{dialog_history}")
])

planner_agent = {
    "person": lambda x: x["person"],
    "test_info": lambda x: x["test_info"],
    "full_plan": lambda x: plans2prompt(x["full_plans"]),     # полный исходный план
    "dialog_history": lambda x: dialog2prompt(x["messages"]),
} | planner_prompt | llm.with_structured_output(planner_schema)


In [69]:
class GraphState(TypedDict):
    person: str
    test_info: str
    full_plans: list[dict[str, str]]  # исходный полный план
    plans: list[dict[str, str]]       # актуальные темы (пересобираются планировщиком)
    messages: list[dict[str, str]]    # история [{role, content}], где у assistant контент = JSON (advisor_schema)
    done: bool


In [70]:
def node_planner(state: GraphState) -> GraphState:
    # вызвать планировщика
    out = planner_agent.invoke({
        "person": state["person"],
        "test_info": state["test_info"],
        "full_plans": state["full_plans"],
        "messages": state["messages"],
    })
    # пересобираем актуальные планы: текущая + оставшиеся (по индексу)
    idx = out["current_plan_index"]
    full = state["full_plans"]
    idx = min(idx, len(full)-1) if full else 0
    new_plans = full[idx:] if idx < len(full) else []

    state["plans"] = new_plans
    state["done"] = bool(out["should_end_dialog"])
    # можно добавить служебное сообщение в историю (по желанию)
    planner_note = {
        "role": "assistant",
        "content": f"[PLAN_STATE] idx={idx}; active='{out['active_topic']}'; "
                   f"upcoming={out['upcoming_topics']}; done={state['done']}; "
                   f"remark={out['assistant_comment']}"
    }
    state["messages"] = state["messages"] + [planner_note]
    return state

def node_examiner(state: GraphState) -> GraphState:
    # вызвать экзаменатора (идентичная схема advisor_schema)
    out = examiner_agent.invoke({
        "person": state["person"],
        "test_info": state["test_info"],
        "plans": state["plans"],      # актуальная + будущие темы
        "messages": state["messages"]
    })
    # кладём в историю user/assistant согласно вашему формату
    state["messages"] = state["messages"] + [{
        "role": "assistant",
        "content": json_dumps(out)  # сохраняем structured output как текст
    }]
    # если экзаменатор отметил завершение текущей темы — в следующем цикле планировщик сдвинет индекс
    return state


In [73]:
def should_stop(state: GraphState) -> bool:
    # стоп если планировщик сказал done=True ИЛИ у нас пустые планы (все темы раскрыты)
    no_more_topics = len(state.get("plans", [])) == 0
    return state.get("done", False) or no_more_topics

def route_after_planner(state: GraphState) -> Literal["END", "examiner"]:
    return "END" if should_stop(state) else "examiner"


In [74]:
import json
json_dumps = lambda o: json.dumps(o, ensure_ascii=False)

workflow = StateGraph(GraphState)
workflow.add_node("planner", node_planner)
workflow.add_node("examiner", node_examiner)

# старт → planner
workflow.set_entry_point("planner")
# planner → (END | examiner)
workflow.add_conditional_edges(
    "planner",
    route_after_planner,
    {
        "END": END,
        "examiner": "examiner"
    }
)
# examiner → обратно к planner
workflow.add_edge("examiner", "planner")

memory = MemorySaver()  # чекпоинты (по желанию)
app = workflow.compile(checkpointer=memory)
